In [1]:
#------------------------- IMPORT LIBRARIES -------------------------#
import torchvision, torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
import logging
import torch
import torch.nn as nn
import torch.optim as optim
from torch.backends import cudnn
import time
import torchvision
from torchvision import transforms
from torchvision.models import alexnet

# FILE DOVE METTIAMO TUTTE LE FUNZIONI CHE CI SERVONO, PER NON INTASARE IL MAIN
from utils import *

from PIL import Image
import copy

In [2]:
# Uso = [1,8] o range(inizio, fine+1)
# OGNI CLASSE HA 500 IMMAGINI: 250 TRAIN + 250 VALIDATION
# SE VOGLIO UNA SOLO CLASSE NON USARE RANGE() MA mettere solo il numero classes=5
train_dataloader, val_dataloader = get_train_val_dataloader(classes=range(0, 5))

Files already downloaded and verified
Train Dataset: 1250
Valid Dataset: 1250


In [3]:
# Ogni classe ha 100 immagini
test_dataloader  = get_test_dataloader(classes=range(0,100))

Files already downloaded and verified
Test Dataset: 10000


In [4]:
#--------------------------------------------------
# Implemento alexnet per il 1° punto del progetto
#--------------------------------------------------

In [5]:
net = alexnet(pretrained=False) # Loading AlexNet model
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# I need to change this because we have NUM_CLASSES
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 100 outputs

In [6]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [7]:
def train_model(model, criterion, optimizer, scheduler):
    since = time.time()

    val_acc_history = []
    val_loss_history = []
    train_acc_history = []
    train_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(NUM_EPOCHS):
        print('Epoch {}/{}/LR={}'.format(epoch, NUM_EPOCHS - 1, scheduler.get_lr()))
        print('-' * 100)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dl = train_dataloader
            else:
                model.train(False)   # Set model to evaluate mode
                dl = val_dataloader

            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss     += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / len(dl.dataset)
            epoch_acc = running_corrects.double() / len(dl.dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model, train_acc_history, train_loss_history, val_acc_history, val_loss_history

In [8]:
net = net.to(DEVICE)
cudnn.benchmark # Calling this optimizes runtime
model, train_acc_history, train_loss_history, val_acc_history, val_loss_history = train_model(net, criterion, optimizer, scheduler)
model

Epoch 0/29/LR=[0.001]
----------------------------------------------------------------------------------------------------


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small

In [ ]:
# VEDERE COME MAI SE AUMENTO IL NUMERO DI EPOCH LA LOSS INIZIALE AUMENTA
plt.figure(figsize=(20, 10))
plt.plot(range(NUM_EPOCHS), val_loss_history, marker="s", color="blue")
plt.plot(range(NUM_EPOCHS), train_loss_history, marker="s", color="green")
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.xticks(range(0, NUM_EPOCHS, 1))
plt.legend(["validation", "train"])
plt.show()

In [ ]:
# VEDERE COME MAI SE AUMENTO IL NUMERO DI EPOCH LA LOSS INIZIALE AUMENTA
plt.figure(figsize=(20, 10))
plt.plot(range(NUM_EPOCHS), val_acc_history, marker="s", color="blue")
plt.plot(range(NUM_EPOCHS), train_acc_history, marker="s", color="green")
plt.xticks(range(0, NUM_EPOCHS, 1))
plt.yticks(np.arange(0, 1.2, 0.2))
plt.xlabel("Epoch")
plt.ylabel("accuracy")
plt.legend(["validation", "train"])
plt.show()

In [ ]:
model = model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
model.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in test_dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))
print('Test Accuracy: {}'.format(accuracy))